In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tools import clean_data, set_new_names, set_category
import os
import glob


In [2]:
df = pd.read_csv(r'../data/Lista_transakcji_nr_0165421859_200723.csv', sep=';', decimal=',', encoding='windows-1250')

In [3]:
df_clean = clean_data(df.copy())
df = df_clean.copy()

In [5]:
df = set_new_names(df.copy())

In [6]:
df["details_2"].fillna("not set", inplace=True)

In [7]:
df = set_category(df.copy())

In [8]:
df['category'].fillna('other', inplace=True)
df['category'] = df['category'].str.lower()
df['details'] = df['details'].str.lower()

In [10]:
df['account'].fillna('unknown', inplace=True)
df['category'] = df['category'].astype('category')

In [13]:
df.head()

,date,details,title,account,transaction,amount,details_2,category
0,2023-07-20,dawid ludwa,bo tak,54102049550000760201606797,202320197209545076,-50.00,not set,other
3,2023-07-20,"zus centrum obsługi świadczeń dla r, odzin, ul...",świadczenie zus 010000c230722tsw/1014718256 ro...,82102056040000010289963017,202320197203279992,500.00,delkatesy,bills
4,2023-07-19,"ludwa dawid jan, i ludwa anna, edmunda ciećkie...",przelew własny,64105015621000009117462698,202320097202882525,1000.00,konto,other
11,2023-07-18,lidl lukasinskiego nowy sacz pol,płatność kartą 18.07.2023 nr karty 4246xx8188,1915031/19730,202320197304224746,-56.20,lidl,grocery store
12,2023-07-18,piekarnia oracz a. oracz nowy sacz,płatność kartą 18.07.2023 nr karty 4246xx8188,1915031/19730,202320197301037340,-40.57,oracz,grocery store


In [14]:
min(df['date']), max(df['date'])

(Timestamp('2023-01-20 00:00:00'), Timestamp('2023-07-20 00:00:00'))

In [17]:
# get current date
from datetime import date
from datetime import datetime   
now = datetime.now()
today = date.today()

In [24]:
file_name = today.strftime("%d%m%Y") + now.strftime("%H%M%S")

In [25]:
file_name

'30012024143053'